In [ ]:
!python -V



# Imports



In [ ]:
import sys
#!{sys.executable} -m pip install pynrrd
#!{sys.executable} -m pip install tensorflow
#!{sys.executable} -m pip install opencv-python
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install matplotlib
#!{sys.executable} -m pip install tdqm
#!{sys.executable} -m pip install scikit-image
#!{sys.executable} -m pip install scikit-learn


In [ ]:
import numpy as np
import nrrd
import os
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from tqdm import trange

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
import numpy as np
from skimage.transform import resize
from sklearn.model_selection import train_test_split

# EDA functions

In [ ]:
def show_slice_window(slice, level, window):

    """
    Permite ajustar nivel y ancho de ventana para mejorar contraste de la imagen.
    input: imagen np.array 2D.
    output: imagen np.array 2D ajustada.
   """
    max = level + window/2
    min = level - window/2
    slice = slice.clip(min,max)
    return(slice)

In [ ]:

def visualizeImgMask(img, mask, index, pet):

    """
  Función para visualizar las imágenes, máscaras y fusión.
  INPUT: imágenes y máscaras como numpy array 3D (numero de imagenes, alto, ancho),
  index corresponde al número de la imágen a visualizar.
  PET: True or False. Si es True se visualiza en inverso (cmap=gray_r),
  sólo por un tema de convención de la visualización del PET.
    """

    if(pet == 1):
        f = plt.figure(figsize=(10,10), frameon=True)
        f.add_subplot(1, 3, 1)
        plt.title("PET")
        plt.imshow(cv2.rotate(img[index,:,:],cv2.ROTATE_90_CLOCKWISE), cmap='gray_r')
        f.add_subplot(1, 3, 2)
        plt.title("Mask")
        plt.imshow(cv2.rotate(mask[index,:,:],cv2.ROTATE_90_CLOCKWISE), cmap='Blues')
        f.add_subplot(1, 3, 3)
        plt.title("Fusion")
        plt.imshow(cv2.rotate(img[index,:,:],cv2.ROTATE_90_CLOCKWISE), cmap='gray_r')
        plt.imshow(cv2.rotate(mask[index,:,:],cv2.ROTATE_90_CLOCKWISE), cmap='gnuplot', alpha= 0.6)
        plt.show()

    else:
        f = plt.figure(figsize=(10,10), frameon=True)
        f.add_subplot(1, 3, 1)
        plt.title("CT")
        plt.imshow(cv2.rotate(img[index,:,:],cv2.ROTATE_90_CLOCKWISE), cmap='gray')
        f.add_subplot(1, 3, 2)
        plt.title("Mask")
        plt.imshow(cv2.rotate(mask[index,:,:],cv2.ROTATE_90_CLOCKWISE), cmap='Blues')
        f.add_subplot(1, 3, 3)
        plt.title("Fusion")
        plt.imshow(cv2.rotate(img[index,:,:],cv2.ROTATE_90_CLOCKWISE), cmap='gray')
        plt.imshow(cv2.rotate(mask[index,:,:],cv2.ROTATE_90_CLOCKWISE), cmap='gnuplot', alpha= 0.6)
        plt.show()


In [ ]:

def extractImages(data_img, data_mask):

    """
  Función que extrae sólo las imágenes de los niveles que contienen segmentación.
  INPUT: volumen de imágenes y máscaras numpy array 3D (numero de imagen, alto, ancho), directo de la
  carga del archivo .nrrd.
  Busca las máscaras con segmentaciones y extrae los cortes de estos niveles.
  OUTPUT: devuelve np.array 3D con las imágenes y máscaras sólo de los niveles del tumor.
    """

    images = []
    masks = []
    positive_slices = []

    for i in trange(data_img.shape[2]):
        segmentation = data_mask[:,:,i]
        if (np.sum(segmentation)>0):
            positive_slices.append(i)

        ## Extrae las imágenes sólo con segmentación - tumor
    for axial in positive_slices:
        segment = data_mask[:,:,axial]
        ct = data_img[:,:,axial]
        images.append(ct)
        masks.append(segment)

    images = np.array(images)
    masks = np.array(masks)
    return(images, masks)


def roiExtraction (img,mask,margin):
    """
  Función para extraer el ROI donde se encuentra el tumor en una imagen.
  INPUT: imagen y máscaras numpy array 2D.
  margin: corresponde al número de pixeles como margen por fuera de los pixeles de la máscara.
  OUTPUT: Devuelve el ROI.
  """

    roi_extract = []

    for i in range(mask.shape[0]):
        img_instance = img[i].copy()
        mask_instance = mask[i].copy()
        index = np.where(mask_instance)
        roi = img_instance[np.unique(index[0])[0]-margin:np.unique(index[0])[-1]+margin, np.unique(index[1])[0]-margin: np.unique(index[1])[-1]+margin]
        roi_extract.append(roi)

    roi_extract = np.array(roi_extract, dtype='object')
    return(roi_extract)


# Data Set Generation


In [ ]:
# PATHs SM (pet, body, torax3d)

#model_name = "pet"
model_name = "body"
#model_name = "torax3d"

path_img_n = f'NRRD/EGFR-/{model_name}/image'
path_mask_n = f'NRRD/EGFR-/{model_name}/label'
############################################
path_img_p = f'NRRD/EGFR+/{model_name}/image'
path_mask_p = f'NRRD/EGFR+/{model_name}/label'

In [ ]:
# PATHs ST (pet, body, torax3d)

#model_name = "pet"
#model_name = "ct"
#model_name = "chest_ct"
#
#path_img_n = f'NNRD-ST/data/AMCEGFR-/image/{model_name}'
#path_mask_n = f'NNRD-ST/data/AMCEGFR-/label/{model_name}'
#
#############################################
#
#path_img_p = f'NNRD-ST/data/AMCEGFR+/image/{model_name}'
#path_mask_p = f'NNRD-ST/data/AMCEGFR+/label/{model_name}'

In [ ]:
patient_list_n = []

for i in range (len(os.listdir(path_img_n))):
    if (os.listdir(path_img_n)[i][-5:] == '.nrrd'):
        patient_list_n.append(os.listdir(path_img_n)[i])
print(patient_list_n)

labels_n =[0 for _ in range(len(patient_list_n))]

#####################################################################

patient_list_p = []

for i in range (len(os.listdir(path_img_p))):
    if (os.listdir(path_img_p)[i][-5:] == '.nrrd'):
        patient_list_p.append(os.listdir(path_img_p)[i])
print(patient_list_p)

labels_p = [1 for _ in range(len(patient_list_p))]

In [ ]:
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(patient_list_n, labels_n, test_size=0.1, random_state=42)
#X_train_n, X_val_n, y_train_n, y_val_n = train_test_split(X_train_v_n, y_train_v_n, test_size=0.1, random_state=42)

X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(patient_list_p, labels_p, test_size=0.1, random_state=42)
#X_train_p, X_val_p, y_train_p, y_val_p = train_test_split(patient_list_p, labels_p, test_size=0.2, random_state=42)

X_train_f = X_train_n + X_train_p
y_train_f = y_train_n + y_train_p

#X_val_f = X_val_n + X_val_p
#y_val_f = y_val_n + y_val_p

X_test_f = X_test_n + X_test_p
y_test_f = y_test_n + y_test_p

In [ ]:
%%capture
## Lectura de los archivos nrrd con sus máscaras, generación de imágenes y obtención del roi (region of interest)

margin = 4

def data_gen(patient_list, labels, model, margin = 4):
    data_list = []
    data_label = []
    patient_id = [s[:6] for s in patient_list]    
    path_img = ''
    path_mask = ''

    for id, label in zip(patient_id, labels):
        if label:
            path_img = path_img_p
            path_mask = path_mask_p     
        else:
            path_img = path_img_n
            path_mask = path_mask_n

        data_tumour,_ = nrrd.read( f'{path_img}/{id}_{model}_image.nrrd')
        mask_tumour,_ = nrrd.read(f'{path_mask}/{id}_{model}_segmentation.nrrd')
        img_tumour, mask_tumour = extractImages(data_tumour, mask_tumour)
        roi_tumour = roiExtraction(img_tumour, mask_tumour, margin)
        for slice in range(len(roi_tumour)):
                data_label.append(label)
                data_list.append(roi_tumour[slice])    
    return data_list, data_label

X_train, y_train = data_gen(X_train_f, y_train_f, model_name)
#X_val, y_val = data_gen(X_val_f, y_val_f, model_name)
X_test, y_test = data_gen(X_test_f, y_test_f, model_name)


In [ ]:
import numpy as np
from skimage.transform import resize
from sklearn.model_selection import train_test_split

def resize_data(df):

    target_input_shape = (32, 32, 1) 

    resized_images = []

    for image in df:
        resized_image = resize(image, target_input_shape, mode='reflect')
        resized_images.append(resized_image)

    data_resized = np.asarray(resized_images).astype(np.float32)

    data_resized = tf.convert_to_tensor(data_resized, dtype=tf.float32)

    return data_resized

In [ ]:
X_train

In [ ]:
X_train_resized = resize_data(X_train)
#X_val_resized = resize_data(X_val)
X_test_resized = resize_data(X_test)

In [ ]:
print(X_train_resized.shape)
#print(X_val_resized.shape)
print(X_test_resized.shape)

In [ ]:
y_train = np.array(y_train)
#y_val = np.array(y_val)
y_test = np.array(y_test)
 

# CNN Implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models


def create_cnn(input_shape):
    model = models.Sequential()

    ### Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(layers.BatchNormalization()) 
    model.add(layers.Activation('relu')) 
    model.add(layers.MaxPooling2D((2, 2)))
    #tf.keras.regularizers.L2(0.01) en todas las capas
    #tf.keras.optimizers.AdamW()
    #tf.keras.optimizers.schedules.CosineDecay(
    #
    model.add(layers.Conv2D(64, (3, 3)))
    model.add(layers.BatchNormalization()) 
    model.add(layers.Activation('relu')) 
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3)))
    model.add(layers.BatchNormalization()) 
    model.add(layers.Activation('relu')) 

    ### Fully connected layer
    model.add(layers.Flatten())
    model.add(layers.Dense(64))   
    model.add(layers.BatchNormalization()) 
    model.add(layers.Activation('relu'))  

    ### Output Layer with Sigmoid activation for binary cross-entropy
    model.add(layers.Dense(1, activation='sigmoid'))

    ### Compile the model with binary cross-entropy loss function
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model


# input dimensions
input_shape = (32, 32, 1)


model = create_cnn(input_shape)
model.summary()

In [ ]:
batch_size = 32  
num_epochs = 50


history = model.fit(
    X_train_resized, 
    y_train, 
    epochs=num_epochs, 
    steps_per_epoch=len(X_train_resized) // batch_size, 
    validation_data=(X_test_resized, y_test),
    verbose=1)

In [ ]:
results = model.evaluate(X_test_resized, y_test, batch_size=16)

In [ ]:
#!mkdir -p saved_model
#model.save('saved_model/'+model_name)

In [ ]:
history_dict = history.history
print(history_dict.keys())

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title(f'Training and validation accuracy - for {model_name}')
plt.legend()

plt.savefig(f'results/{model_name}-acc.png')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title(f'Training and validation loss - for {model_name}')
plt.legend()

plt.savefig(f'results/{model_name}-loss.png')

plt.show()